# Fit hyper-parameters

In [1]:
# Setup ipython environment
%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib.pyplot import *
from numpy import *
from positive import *
from nrutils import scsearch, gwylm
from glob import glob
import pwca
from pwca import determine_data_fitting_region,pwca_catalog,metadata_dict

(positive)>> Applying custom matplotlib settings.
(init)>> Warm greetings from Welsh Puck (aka pwca).
(pwca.core)>> Catalog of calibration runs stored to "pwca.pwca_catalog"
(pwca.core)>> Metadata dictionary for calibration runs stored to "pwca.metadata_dict"


## This is notebook is only for scratch code and prototyping

In [135]:
# Load and unpuack physical parameter space
raw_domain = loadtxt('/Users/book/KOALA/puck/ll/data/version2/fit_intial_binary_parameters.txt')
theta,m1,m2,eta,delta,chi_eff,chi_p,chi1,chi2 = raw_domain.T


# Define desired model domain variables and array 
u = cos(theta)
model_domain = array( [ u, eta, chi_eff, chi_p ] ).T

# Load and unpuack physical parameter space -- dphi
dphi_range = loadtxt('/Users/book/KOALA/puck/ll/data/version2/fit_opt_dphase_parameters.txt')
nu4,nu5,nu6 = dphi_range.T

# Load and unpuack physical parameter space -- amp
amp_range = loadtxt('/Users/book/KOALA/puck/ll/data/version2/fit_opt_amplitude_parameters.txt')
mu1, mu2, mu3, mu4 = amp_range.T

# labels={'python':['nu4',('u', 'eta', 'chi_eff', 'chi_p'),''],'latex':['nu4',(r'u', r'\eta', r'\chi_s', r'\chi_p'),'']}

In [79]:
# nu4
foo = gmvpfit( model_domain, nu4,fitatol=0.002,verbose=True,maxdeg_list=[3,2,1,1],center=True)
foo.plot();
# nu5
foo = gmvpfit( model_domain, nu5,fitatol=0.002,verbose=True,maxdeg_list=[3,3,1,1],center=True)
foo.plot();
# nu6
foo = gmvpfit( model_domain, nu6,fitatol=0.002,verbose=True,maxdeg_list=[3,3,1,1],center=False)
foo.plot();

In [ ]:
# mu1 
# ---

# possibly overfit
foo = gmvpfit( model_domain, mu1,fitatol=0.004,verbose=True,maxdeg_list=[2,3,1,2],center=False,temper=False)
foo.plot();

# mu2
# ---

# not overfit
foo = gmvpfit( model_domain, mu2,fitatol=0.004,verbose=True,maxdeg_list=[2,2,2,2],center=True)
foo.plot();

# mu3
# ---

# not overfit
foo = gmvpfit( model_domain, mu3,fitatol=0.004,verbose=True,maxdeg_list=[2,2,2,2],center=True)
foo.plot();

# mu4
# ---

# not overfit
foo = gmvpfit( model_domain, mu4,fitatol=0.004,verbose=True,maxdeg_list=[2,2,2,2],center=False)
foo.plot();

